In [3]:
import sys
%cd variational_inference_for_longitudinal_data/
sys.path.append('diffusion/stable_diffusion/')
sys.path.append('diffusion/stable_diffusion/model/')
sys.path.append('lib/src/')
import torch
from torch import nn
import os
from matplotlib import pyplot as plt
from sklearn_extra.cluster import KMedoids


from diffusion.stable_diffusion.latent_diffusion import MyLatentDiffusion, LitLDM
from diffusion.stable_diffusion.model.autoencoder import Autoencoder
from diffusion.stable_diffusion.model.clip_embedder import CLIPTextEmbedder
from diffusion.stable_diffusion.model.unet import UNetModel, _test_time_embeddings

from lib.src.pythae.models import VAE
from lib.src.pythae.models import LVAE_IAF, LVAE_IAF_Config, AutoModel
from lib.src.pythae.models.nn.base_architectures import BaseDecoder
from lib.scripts.utils import Encoder_Chairs,Decoder_Chairs, My_Dataset, My_MaskedDataset, make_batched_masks

def load_config_unet(config):
    return UNetModel(
        in_channels=config['in_channels'],
        out_channels=config['out_channels'],
        channels=config['channels'],
        n_res_blocks=config['n_res_blocks'],
        attention_levels=config['attention_levels'],
        channel_multipliers=config['channel_multipliers'],
        n_heads=config['n_heads'],
    )

%reload_ext autoreload
%autoreload 2

!nvidia-smi

[Errno 2] No such file or directory: 'variational_inference_for_longitudinal_data/'
/home/mtayebje/variational_inference_for_longitudinal_data


ModuleNotFoundError: No module named 'sklearn_extra'

In [5]:
train_data = torch.load(os.path.join('sprites/Sprites_train.pt'))[:-1000]
eval_data = torch.load(os.path.join('sprites/Sprites_train.pt'), map_location="cpu")[-1000:]
#test_data = torch.load(os.path.join('lib/my_data/sprites/Sprites_test.pt'), map_location="cpu")

print(train_data.shape)
train_data = train_data.permute(0, 1, 4, 2, 3)
eval_data = eval_data.permute(0, 1, 4, 2, 3)
#test_data = test_data.permute(0, 1, 4, 2, 3)
print(train_data.shape)

train_seq_mask = torch.ones(train_data.shape[:2], requires_grad=False).type(torch.bool)
eval_seq_mask = torch.ones(eval_data.shape[:2], requires_grad=False).type(torch.bool)
#test_seq_mask = torch.ones(test_data.shape[:2], requires_grad=False).type(torch.bool)
train_pix_mask = torch.ones_like(train_data, requires_grad=False).type(torch.bool)
eval_pix_mask = torch.ones_like(eval_data, requires_grad=False).type(torch.bool)
#test_pix_mask = torch.ones_like(test_data, requires_grad=False).type(torch.bool)

train_dataset = My_MaskedDataset(train_data, train_seq_mask, train_pix_mask)
eval_dataset = My_MaskedDataset(eval_data, eval_seq_mask, eval_pix_mask)
#test_dataset = My_MaskedDataset(test_data, test_seq_mask, test_pix_mask)

torch.Size([8000, 8, 64, 64, 3])
torch.Size([8000, 8, 3, 64, 64])


In [5]:
# encoder = Encoder_Chairs(config)
# decoder = Decoder_Chairs(config)
# vae = LVAE_IAF(config, encoder, decoder)

device = 'cuda'
vae = VAE.load_from_folder('pre-trained_vae/VAE_training_2024-05-22_14-43-10/final_model').to(device)
vae.eval()

in_channels = 3
out_channels = 3
channels = 32
n_res_blocks = 2
attention_levels = [2]
channel_multipliers = (1, 2, 4)
n_heads = 16

unet_config = {
    'in_channels': in_channels,
    'out_channels': out_channels,
    'channels': channels,
    'n_res_blocks': n_res_blocks,
    'attention_levels': attention_levels,
    'channel_multipliers': channel_multipliers,
    'n_heads': n_heads,
}

unet = load_config_unet(unet_config)

latent_scaling_factor = 1
n_steps = 1000
linear_start =  0.00085
linear_end = 0.05


latent_diffusion = MyLatentDiffusion(unet, None, latent_scaling_factor, n_steps, linear_start, linear_end)
print('Number of parameters in the diffusion model: ', sum(p.numel() for p in latent_diffusion.parameters() if p.requires_grad))

model = LitLDM.load_from_checkpoint('pre-trained_LDM/version_15/checkpoints/epoch=99-step=8000.ckpt', ldm = latent_diffusion).to('cuda')
diffusion = model.ldm



Number of parameters in the diffusion model:  5721539


In [ ]:
def retrieveG(vae, train_loader):
    mu = []
    log_var = []
    with torch.no_grad():
        for _ , x in enumerate(train_loader):

            data = x.data

            out = vae.encoder(data.to(device))

            mu_data, log_var_data = out.embedding, out.log_covariance

            mu.append(mu_data)
            log_var.append(log_var_data)

    mu = torch.cat(mu)
    log_var = torch.cat(log_var)

    print('Running Kmedoids')
    kmedoids = KMedoids(n_clusters=100).fit(mu.detach().cpu())
    medoids = torch.tensor(kmedoids.cluster_centers_).to(device)
    centroids_idx = kmedoids.medoid_indices_ #




In [2]:
def log_p_j_hat(j, z, zT_samples, alpha_bars, tau = 125):
    """

    """
    alpha_bar_j = alpha_bars[j*tau]

    log_density = torch.sum( (z - zT_samples)**2 / (2 * (1 - alpha_bar_j)**2), dim=-1)

    return log_density.mean()




In [18]:
zT_samples = torch.load('zT_samples.pt').to('cpu')
z = torch.randn(1, 192)
alpha_bars = 0.5*torch.ones(1000)
log_p_j_hat(1, z, zT_samples, alpha_bars)


tensor(374.5630)

## TO DO

#ergonomiser le Riemaniann sampling
- inspecter LVIAF
- train !

Possible pain points : le diffusion backward très bof (at least dans les intemédiaires). Après c'est du sampling... wait and see


Voir s'il faut baisser la dim latent